In [14]:
import numpy as np
import numpy
from numpy.linalg import inv
from fractions import Fraction

# Notation setup

n = 5
print("\nG = SL(5)\n")

''' Define basis beta and varpi '''
beta = numpy.zeros((n-1,n))
varpi = numpy.zeros((n-1,n))
for i in range(n-1):
    for j in range(n):
        if j == i:
            beta[i,j] = 1
        elif j == i + 1:
            beta[i,j] = -1
        else:
            beta[i,j] = 0
    for j in range(n):
        if j <= i:
            varpi[i,j] = (n-i-1)/float(n)
        else: 
            varpi[i,j] = -(i+1)/float(n)

PositiveRoots = []
for i in range(n-1):
    for j in range(i+1, n):
        vector = numpy.zeros(n)
        vector[i] = 1
        vector[j] = -1
        PositiveRoots.append(vector)

def theta(basis):
    '''This function takes an ordered basis and flips the order.'''
    temp = numpy.copy(basis)
    length = len(basis)
    for i in range(length):
        temp[i] = basis[length-1-i]
    return temp

def V_to_a(basis, vector):
    ''' Express vector as a linear combination of basis elements, 
            returns the (n-1) coefficients
            i.e., V to aaa 
            inputs: 
            hopefully basis is either beta or varpi
            vector is a size n vector with all coordinates adding up to zero, that is, an element of V
            outputs: 
            array of coordinate vectors of vector with respect to basis, size (n-1)
            '''
    basis = basis[:,:-1]
    basis = numpy.transpose(inv(basis))
    vector = vector[:-1]
    return numpy.dot(basis,vector)

def last_coordinate(vector):
    ''' This appends the last coordinate to the vector so the sum of all coordinates is zero '''
    return numpy.append(vector, -numpy.sum(vector))

def change_of_basis(basis_1, basis_2):
    ''' This returns the change-of-basis matrix. 
    basis_1 is the original basis and basis_2 is the target basis 
    This is a nice solution actually'''
    A = None
    for i in range(len(basis_1)):
        if i == 0: 
            A = V_to_a(basis_1, basis_2[i])
        else:
            A = numpy.concatenate((A, V_to_a(basis_1, basis_2[i])), axis=0)
    return numpy.reshape(A,(n-1, n-1)) # maybe return transpose instead? 

# def w_vector(vector, sigma):
#     ''' Permutes the coordinates of vector according to sigma '''
#     sigma = numpy.append(sigma, sigma[0])
#     new_vector = numpy.copy(vector)
#     for i, item in enumerate(sigma[:-1]):
#         new_vector[item] = vector[sigma[i+1]]
#     return new_vector

def w_vector(mu, sigma):
    ''' Permutes the coordinates of vector in V according to sigma '''
    new_mu = numpy.copy(mu)
    for i, item in enumerate(sigma):
        new_mu[item] = mu[i]
    return new_mu

def w(matrix, sigma):
    ''' Permutes every vector in the matrix according to sigma '''
    new_matrix = numpy.copy(matrix)
    for i in range(len(matrix)):
        new_matrix[i] = w_vector(matrix[i], sigma)
    return new_matrix
        
# vector = (Id - theta.w)(varpi)
# vector = varpi - theta (w(varpi, [0,1]))
# change_of_basis(beta, vector)

def a_to_V(basis, coefficients):
    '''Returns vector in my hyperplane, given coefficients
        i.e., V \to \aaa '''
    vector = numpy.zeros(len(basis)+1)
    for i in range(len(basis)):
        vector = vector + coefficients[i] * numpy.array(basis[i])
    return vector

def sanitize(vector):
    for i in range(len(vector)):
        vector[i] = round(vector[i])
    return vector

def mymatrix(coefficients, sigma): # coefficients = rows of identity matrix
    mylambda = a_to_V(varpi, coefficients)
    wlambda = w_vector( mylambda, sigma)
    vector = V_to_a(theta(varpi), wlambda)
    vector = coefficients - vector
    vector = a_to_V(varpi, vector) 
    return V_to_a(beta, vector)

def FL_matrix(coefficients, sigma): # coefficients = rows of identity matrix
    mylambda = a_to_V(varpi, coefficients)
    wlambda = w_vector( mylambda, sigma)
    vector = V_to_a(varpi, wlambda)
    vector = coefficients - vector
    vector = a_to_V(varpi, vector) 
    return V_to_a(beta, vector)

def error_matrix(coefficients, sigma): # coefficients = rows of identity matrix
    mylambda = a_to_V(varpi, coefficients)
    wlambda = w_vector( mylambda, sigma)
    return V_to_a(beta, wlambda) - V_to_a(theta(beta),wlambda)

w = (4,3,2,1) # w = (sigma(0), sigma(1), ...) really
print "\n---------------------------------"
print "w is ", w
print "---------------------------------"
print "the FL matrix for lambda - w(lambda) is:"
finalmatrix = []
for row in numpy.identity(n-1):
    print sanitize(FL_matrix(row, w))
    
print "\n-----------------------------------"
print "the error matrix for (1 - theta)(w(lambda)) is:"
finalmatrix = []
for row in numpy.identity(n-1):
    print error_matrix(row, w)

print "\n-----------------------------------"
print "the matrix for lambda - theta.w(lambda) is:"
finalmatrix = []
for row in numpy.identity(n-1):
    print mymatrix(row, w)

print "\n-----------------------------------"


''' Do not forget this: If the output matrix is (a_{i,j}) then coefficient of beta_1 is a_11 c_1 + a_12 c_2 etc. '''


G = SL(5)


---------------------------------
w is  (4, 3, 2, 1)
---------------------------------
the FL matrix for lambda - w(lambda) is:
[-0. -0. -0. -0.]
[-0.  1.  1.  0.]
[-0.  1.  1.  0.]
[ 0.  0.  0.  0.]

-----------------------------------
the error matrix for (1 - theta)(w(lambda)) is:
[ 0.6  0.2 -0.2 -0.6]
[ 0.2  0.4 -0.4 -0.2]
[-0.2 -0.4  0.4  0.2]
[-0.6 -0.2  0.2  0.6]

-----------------------------------
the matrix for lambda - theta.w(lambda) is:
[ 0.6  0.2 -0.2 -0.6]
[ 0.2  1.4  0.6 -0.2]
[-0.2  0.6  1.4  0.2]
[-0.6 -0.2  0.2  0.6]

-----------------------------------


' Do not forget this: If the output matrix is (a_{i,j}) then coefficient of beta_1 is a_11 c_1 + a_12 c_2 etc. '

In [8]:
from itertools import permutations #for winv in permutations(range(n)):
import math # for factorial         
import numpy
from numpy.linalg import inv

n = 5
def perm_to_matrix(winv): 
    '''winv is a permutation of length n. return the matrix of winv wrt beta'''
    matrix = []
    for i in range(n-1):
        temp = np.zeros(n)
        for j in range(n):
            temp[j] = beta[i][winv[j]]
        matrix.append(V_to_a(beta, temp))
    matrix = np.array(matrix)
    return matrix.T

beta = numpy.zeros((n-1,n))
varpi = numpy.zeros((n-1,n))
for i in range(n-1):
    for j in range(n):
        if j == i:
            beta[i,j] = 1
        elif j == i + 1:
            beta[i,j] = -1
        else:
            beta[i,j] = 0
    for j in range(n):
        if j <= i:
            varpi[i,j] = (n-i-1)/float(n)
        else: 
            varpi[i,j] = -(i+1)/float(n)

# mylambda in original coordinates (varpi), row vector
mylambda = np.array([1]*(n-1)) 
# newlambda wrt new coordinates (beta), column vector
newlambda = np.dot(mylambda, change_of_basis(beta, varpi)).reshape(n-1,1) 

mytheta = []
for i in range(n-1):
    row = np.zeros(n-1)
    for j in range(n-1):
        if i+j == n-2:
            row[j] = 1
    mytheta.append(row)

def myfunc(winv, mylambda):
    newlambda = np.dot(mylambda, change_of_basis(beta, varpi)).reshape(n-1,1) 
    out = newlambda - np.dot(mytheta, np.dot(perm_to_matrix(winv), newlambda))
    fl = newlambda - np.dot(perm_to_matrix(winv), newlambda)
    error = np.dot((np.eye(n-1) - mytheta), np.dot(perm_to_matrix(winv), newlambda))
    return out.T #, fl.T, error.T

mybiglist = []
MAX = 3**(n-1)
for num in range(MAX):
    lst = []
    ntemp = num
    for j in range(n-1):
        digit = ntemp % 3
        ntemp = (ntemp - digit) / 3
        lst.append(digit+1)
    mybiglist.append(lst)
    
def main():
    successful = []
#    tmpcounter = 0
    for winv in permutations(range(n)):
#        tmpcounter += 1
#        print tmpcounter
        #print 'trying for winv =', winv
        for mylambda in mybiglist:
            output = myfunc(winv, mylambda)[0]
            mybool = True
            for i in range(n-1):
                # construct winv(varpi_i) and check if it is the same as varpi_i
                # if yes, don't check the output[i], otherwise check. 
                temp = np.zeros(n)
                for j in range(n):
                    temp[j] = varpi[i][winv[j]]
                if not np.array_equal(temp, varpi[i]):
                    if output[i] <= 0.0001:
                        mybool = False
                        break
            if mybool:
#                print "Success for winv = ", winv, "with lambda = ", mylambda
                successful.append((winv, mylambda))
                break
    return successful

success = main()

print("Total successes: " + len(success) + " out of " + math.factorial(n))

TypeError: cannot concatenate 'str' and 'int' objects

In [9]:
'''This code finds the set Delta(winv) = \Delta_0^{Q(w)} given w. '''
def deltaofw(winv):
    mylist = []
    for i in range(n-1):
        temp = np.zeros(n)
        for j in range(n):
            temp[j] = varpi[i][winv[j]]
        if not np.array_equal(temp, varpi[i]):
            mylist.append(i)
    return mylist

'''This function finds length of winv'''
def length(winv):
    count = 0
    for root in PositiveRoots:
        if numpy.sum(V_to_a(beta, w_vector(root, winv))) < 0:
            count = count + 1
    return count

w0 = [n-i-1 for i in range(n)]
w0winv = [i for i in range(n)]

# --- Uncomment below section to print --- 
for winv in permutations(range(n)):
     for i in range(n):
         w0winv[i] = w0[winv[i]];
     print "For w = ", winv, "length = ", length(winv), " Delta(w) = ", deltaofw(winv)
     print "w0.winv = ", tuple(w0winv), "length = ", length(w0winv), "Delta(w0 winv)", deltaofw(w0winv), "\n"


For w =  (0, 1, 2, 3, 4) length =  0  Delta(w) =  []
w0.winv =  (4, 3, 2, 1, 0) length =  10 Delta(w0 winv) [0, 1, 2, 3] 

For w =  (0, 1, 2, 4, 3) length =  1  Delta(w) =  [3]
w0.winv =  (4, 3, 2, 0, 1) length =  9 Delta(w0 winv) [0, 1, 2, 3] 

For w =  (0, 1, 3, 2, 4) length =  1  Delta(w) =  [2]
w0.winv =  (4, 3, 1, 2, 0) length =  9 Delta(w0 winv) [0, 1, 2, 3] 

For w =  (0, 1, 3, 4, 2) length =  2  Delta(w) =  [2, 3]
w0.winv =  (4, 3, 1, 0, 2) length =  8 Delta(w0 winv) [0, 1, 2, 3] 

For w =  (0, 1, 4, 2, 3) length =  2  Delta(w) =  [2, 3]
w0.winv =  (4, 3, 0, 2, 1) length =  8 Delta(w0 winv) [0, 1, 2, 3] 

For w =  (0, 1, 4, 3, 2) length =  3  Delta(w) =  [2, 3]
w0.winv =  (4, 3, 0, 1, 2) length =  7 Delta(w0 winv) [0, 1, 2, 3] 

For w =  (0, 2, 1, 3, 4) length =  1  Delta(w) =  [1]
w0.winv =  (4, 2, 3, 1, 0) length =  9 Delta(w0 winv) [0, 1, 2, 3] 

For w =  (0, 2, 1, 4, 3) length =  2  Delta(w) =  [1, 3]
w0.winv =  (4, 2, 3, 0, 1) length =  8 Delta(w0 winv) [0, 1, 2, 3] 

For 

In [12]:
# In this part of code, I am trying to find the value of \varpi - w \varpi in terms of beta's. 
# If this coefficient is large enough, then I might try to use Shahidi's trick to bound 
# the negative term by the leading one in:
#       \lambda - \theta w \lambda = (\lambda - w \lambda) + w.(\lambda - \theta \lambda)
#   Let's see how goes. 

def flfunc(winv, mylambda):
    newlambda = np.dot(mylambda, change_of_basis(beta, varpi)).reshape(n-1,1) 
    fl = newlambda - np.dot(perm_to_matrix(winv), newlambda)
    return fl.T[0]

def sanitize(vector):
    for i in range(len(vector)):
        vector[i] = round(vector[i])
    return vector

w = (4,3,2,1,0)
for winv in permutations(range(n)):
    for i in range(n-1):
        mylambda = np.array([0]*(n-1)) 
        mylambda[i] = 1
        print "For winv = ", winv, "and varpi",i, ", (varpi-winv varpi) equals ", sanitize(flfunc(winv, mylambda))


For winv =  (0, 1, 2, 3, 4) and varpi 0 , (varpi-winv varpi) equals  [ 0.  0.  0.  0.]
For winv =  (0, 1, 2, 3, 4) and varpi 1 , (varpi-winv varpi) equals  [ 0.  0.  0.  0.]
For winv =  (0, 1, 2, 3, 4) and varpi 2 , (varpi-winv varpi) equals  [ 0.  0.  0.  0.]
For winv =  (0, 1, 2, 3, 4) and varpi 3 , (varpi-winv varpi) equals  [ 0.  0.  0.  0.]
For winv =  (0, 1, 2, 4, 3) and varpi 0 , (varpi-winv varpi) equals  [ 0.  0.  0.  0.]
For winv =  (0, 1, 2, 4, 3) and varpi 1 , (varpi-winv varpi) equals  [ 0.  0.  0. -0.]
For winv =  (0, 1, 2, 4, 3) and varpi 2 , (varpi-winv varpi) equals  [ 0.  0.  0.  0.]
For winv =  (0, 1, 2, 4, 3) and varpi 3 , (varpi-winv varpi) equals  [ 0.  0.  0.  1.]
For winv =  (0, 1, 3, 2, 4) and varpi 0 , (varpi-winv varpi) equals  [ 0.  0.  0.  0.]
For winv =  (0, 1, 3, 2, 4) and varpi 1 , (varpi-winv varpi) equals  [ 0.  0.  0.  0.]
For winv =  (0, 1, 3, 2, 4) and varpi 2 , (varpi-winv varpi) equals  [ 0.  0.  1.  0.]
For winv =  (0, 1, 3, 2, 4) and varpi 3 , (

In [10]:
# Testing the specific case n = 5, w = (23)(45), c_1 = c_3 = c_4 = 1, c_2 = 2. 
# Should pass and passes. 

winv = (0,2,1,4,3)
mylambda = [1,2,1,2]
output = myfunc(winv, mylambda)[0]
mybool = True
for i in range(n-1):
    # construct winv(varpi_i) and check if it is the same as varpi_i
    # if yes, don't check the output[i], otherwise check. 
    temp = np.zeros(n)
    for j in range(n):
        temp[j] = varpi[i][winv[j]]
    print "coordinate ", i, " is ", output[i]
    if not np.array_equal(temp, varpi[i]):
        if output[i] <= 0.0001:
            mybool = False
if mybool:
    print "Success for winv = ", winv, "with lambda = ", mylambda

coordinate  0  is  1.6
coordinate  1  is  0.2
coordinate  2  is  1.8
coordinate  3  is  0.4
Success for winv =  (0, 2, 1, 4, 3) with lambda =  [1, 2, 1, 2]
